In [1]:
%run -i 'src/SPV.py'
from src.data_functions import *
from src.solution_algorithms import *

In [2]:
df = pd.read_csv('data/clean_2017_hourly_normalized.csv')
df.head()

,dataid,localhour,gen,use,pv_system_size,gen_per_kW
0,26,2017-01-01 00:00:00,0.0,1.788817,6.0,0.0
1,26,2017-01-01 01:00:00,0.0,1.223917,6.0,0.0
2,26,2017-01-01 02:00:00,0.0,0.668367,6.0,0.0
3,26,2017-01-01 03:00:00,0.0,0.478667,6.0,0.0
4,26,2017-01-01 04:00:00,0.0,0.416283,6.0,0.0


In [3]:
dataids = obtain_dataid_2017(df)
dataids_array = np.asarray(dataids)
validdata = len(dataids)
T = len(df[df.dataid==26].gen.values)

In [4]:
gen_kw = f_gen_per_kw(df)
load_kw = f_load_kw(df)

1 m$^2$ is 1550 in$^2$

Typical size of solar panel: is 65 in for 39 in = 2535 in$^2$ = 1.6354 m$^2$.

The rated power of a solar panel is typically between 250 to 350W.

To convert we use: gen_per_kw by 0.3 / 1.6354 to get gen_per_m$^2$.

In [5]:
gen_per_m2 = gen_kw*0.3/1.6354

In [6]:
#Investment per m2 is 512.2 $/m2.
#Using an anualized cost with discount rate r=5%, we obtain
CostPVperM2 = 512.2
r = 0.05
years = 20
annuity = CostPVperM2*r/(1-(1+r)**(-years)) #Obtain Annuity 
pi_s = annuity/T #Obtain cost per time step
print('Cost of PV per time step: ' + str(pi_s))

Cost of PV per time step: 0.004695025492240356


In [7]:
firms = validdata #Number of firms
gamma = 0.85 #Factor of NM price
pi_r = 0.18   #Retail Price at 18 cents per kWh.
pi_nm = gamma*pi_r  #Net Metering Price
a_cap_firms = f_cap_firms(gen_per_m2, load_kw) #Cap for firms to not be net producers
a_max_firms = a_cap_firms

In [8]:
investment_standalone = solve_standalone(gen_per_m2, load_kw, dataids, a_max_firms, pi_s, pi_r, pi_nm)

Solving firm 194 (of 194) with dataid 100267


In [9]:
print('\n')
print('Investment decisions:')
print(investment_standalone) #Print investment decisions
print('\n')
print('Percentage of investment per firm (with respect to max_cap)')
print(1-(a_max_firms - investment_standalone)/a_max_firms) #Print percentage of investment of available max cap
print('\n Total Investment of PV in standalone case is '+ str(sum(investment_standalone)) + ' in m2 \n')



Investment decisions:
[ 0.          0.          0.          0.          0.         19.70738606
 61.86978323  0.          0.          0.          0.          1.7283147
  0.          0.          0.         24.45501998  0.         59.86670663
  0.          4.03594149  0.          1.54512289  2.14550516  0.
  0.          3.73349541 49.01041927 22.01660325  0.         10.98888394
  0.          0.          0.          0.          0.         10.83455298
 34.7362296   0.          4.18910604  0.          7.5175913  19.11353186
  0.          2.07246604  0.         21.83347856  2.77866639  0.
 12.71965854 15.75077687 10.46275701  8.73833943  7.70548716  0.
 11.38585394 30.97976477  0.          0.          0.          0.
  0.          1.24104627  0.         12.72706689  0.          0.
 35.96039878  0.          0.          6.83864602  0.          0.
  0.          0.          2.27465711  7.33982538  0.          0.
  0.          0.          0.          0.          0.          7.82713635
  4.8152607

In [10]:
soluc_sharing = solve_sharing_collective(gen_per_m2, load_kw, dataids_array, a_max_firms, pi_s, pi_r)



Iteration: 1
firm removed from S: 9 with dataid 499
Set of firms that invest:
[148, 186, 153, 187, 146, 97, 182, 173, 144, 5, 156, 123, 90, 145, 57, 185, 29, 65, 81, 180, 95, 6, 92, 160, 164, 38, 116, 178, 24, 155, 165, 167, 172, 59, 70, 17, 31, 8, 170, 128, 45, 89, 64, 80, 98, 105, 11, 27, 50, 23, 179, 67, 51, 190, 30, 115, 91, 88, 104, 86, 114, 1, 84, 26, 10, 177, 74, 77, 78, 106, 129, 61, 7, 163, 63, 58, 141, 103, 162, 93, 188, 150, 134, 158, 154, 53, 13, 135, 157, 28, 0, 121, 79, 42, 118, 15]
Set of firms that do not invest:
[35, 54, 183, 37, 112, 130, 111, 137, 3, 124, 75, 48, 113, 52, 119, 16, 41, 102, 159, 55, 117, 36, 122, 66, 72, 94, 100, 133, 12, 140, 82, 151, 139, 191, 71, 87, 44, 14, 120, 21, 175, 143, 2, 132, 142, 193, 138, 39, 49, 181, 20, 99, 149, 108, 166, 126, 192, 22, 4, 161, 176, 174, 19, 83, 43, 125, 136, 33, 76, 68, 47, 40, 69, 184, 85, 34, 127, 107, 25, 46, 73, 60, 147, 189, 169, 152, 18, 101, 62, 32, 110, 131, 96, 168, 171, 109, 56, 9]
Number of times doing the

In [11]:
investment_sharing = firms_investment_sharing(soluc_sharing, a_max_firms, firms)

In [12]:
profit_nopv = utility_profit_no_investment(load_kw, pi_r)
profit_standalone = utility_profit_standalone(gen_per_m2, load_kw, investment_standalone, pi_r, pi_nm)
profit_sharing = utility_profit_sharing(gen_per_m2, load_kw, investment_sharing, pi_r)

Average profit per time slot without PV investment for utility is 50.538087833479416
Average profit per time slot for utility (with standalone PV investment) is 43.93869562342025
Average profit per time slot for utility (with sharing PV investment) is 42.56674389376173


In [13]:
np.sum(pi_r*load_kw)

442410.4208942788

In [75]:
df_data = df.drop(columns=['pv_system_size', 'gen'])

In [76]:
print(df_data[df_data.localhour=='2017-01-01 00:00:00'])

         dataid            localhour       use  gen_per_kW
0            26  2017-01-01 00:00:00  1.788817         0.0
8754         77  2017-01-01 00:00:00  0.551283         0.0
17508        93  2017-01-01 00:00:00  0.916167         0.0
26262       114  2017-01-01 00:00:00  0.552000         0.0
35016       171  2017-01-01 00:00:00  0.456217         0.0
43770       252  2017-01-01 00:00:00  1.218867         0.0
52524       330  2017-01-01 00:00:00  0.627533         0.0
61278       370  2017-01-01 00:00:00  0.862550         0.0
70032       379  2017-01-01 00:00:00  1.167650         0.0
78786       499  2017-01-01 00:00:00  1.040433         0.0
87540       545  2017-01-01 00:00:00  0.639683         0.0
96294       547  2017-01-01 00:00:00  0.538417         0.0
105048      585  2017-01-01 00:00:00  0.177483         0.0
113802      668  2017-01-01 00:00:00  1.298200         0.0
122556      744  2017-01-01 00:00:00  0.226833         0.0
131310      781  2017-01-01 00:00:00  0.290350         0

In [77]:
diff = 1000-194
T = len(gen_per_m2[:,0])
time_data = df[df.dataid==26].localhour.values
for i in range(diff):
    rand_indices = np.random.randint(194, size=2)
    print(rand_indices)
    rand_gen = (gen_kw[:,rand_indices[0]] + gen_kw[:,rand_indices[1]]  )/2
    rand_load = (load_kw[:,rand_indices[0]] + load_kw[:,rand_indices[1]] )/2
    rand_dataid = 990000 + i
    rand_dataid_aux = np.repeat(rand_dataid, T)
    data_aux = {'dataid': rand_dataid_aux, 'localhour': time_data, 'use': rand_load, 'gen_per_kW': rand_gen }
    df_aux = pd.DataFrame(data_aux)
    print(i)
    df_data = pd.concat([df_data,df_aux], sort=False)
    print(i)
    clear_output()
print(df_data[df_data.localhour=='2017-01-01 00:00:00'])   


        dataid            localhour       use  gen_per_kW
0           26  2017-01-01 00:00:00  1.788817         0.0
8754        77  2017-01-01 00:00:00  0.551283         0.0
17508       93  2017-01-01 00:00:00  0.916167         0.0
26262      114  2017-01-01 00:00:00  0.552000         0.0
35016      171  2017-01-01 00:00:00  0.456217         0.0
43770      252  2017-01-01 00:00:00  1.218867         0.0
52524      330  2017-01-01 00:00:00  0.627533         0.0
61278      370  2017-01-01 00:00:00  0.862550         0.0
70032      379  2017-01-01 00:00:00  1.167650         0.0
78786      499  2017-01-01 00:00:00  1.040433         0.0
87540      545  2017-01-01 00:00:00  0.639683         0.0
96294      547  2017-01-01 00:00:00  0.538417         0.0
105048     585  2017-01-01 00:00:00  0.177483         0.0
113802     668  2017-01-01 00:00:00  1.298200         0.0
122556     744  2017-01-01 00:00:00  0.226833         0.0
131310     781  2017-01-01 00:00:00  0.290350         0.0
140064     890

In [79]:
df_data.to_csv('full_randomized_2017_hourly_normalized_v2.csv', sep=',', index=False)

In [78]:
df_data.shape

(8754000, 4)

In [63]:
i = 0
rand_indices = np.random.randint(194, size=3)
rand_gen = (gen_kw[:,rand_indices[0]] + gen_kw[:,rand_indices[1]]  + gen_kw[:,rand_indices[2]] )/3
rand_load = (load_kw[:,rand_indices[0]] + load_kw[:,rand_indices[1]] + load_kw[:,rand_indices[1]])/3
rand_dataid = 990000 + i
rand_dataid_aux = np.repeat(rand_dataid, T)
time_data = df[df.dataid==26].localhour.values
data_aux = {'dataid': rand_dataid_aux, 'localhour': time_data, 'use': rand_load, 'gen_per_kW': rand_gen }
df_aux = pd.DataFrame(data_aux)

In [41]:
df_data = pd.concat([df_data,df_aux], sort=False)
print(df_data)

      dataid            localhour       use  gen_per_kW
0         26  2017-01-01 00:00:00  1.788817    0.000000
1         26  2017-01-01 01:00:00  1.223917    0.000000
2         26  2017-01-01 02:00:00  0.668367    0.000000
3         26  2017-01-01 03:00:00  0.478667    0.000000
4         26  2017-01-01 04:00:00  0.416283    0.000000
5         26  2017-01-01 05:00:00  0.357633    0.000000
6         26  2017-01-01 06:00:00  0.383550    0.000000
7         26  2017-01-01 07:00:00  0.483333    0.005814
8         26  2017-01-01 08:00:00  0.413217    0.051969
9         26  2017-01-01 09:00:00  0.388917    0.237289
10        26  2017-01-01 10:00:00  0.357817    0.453742
11        26  2017-01-01 11:00:00  0.550117    0.495628
12        26  2017-01-01 12:00:00  0.567700    0.553214
13        26  2017-01-01 13:00:00  0.552417    0.543053
14        26  2017-01-01 14:00:00  1.209267    0.470700
15        26  2017-01-01 15:00:00  2.704533    0.342294
16        26  2017-01-01 16:00:00  1.537783    0

In [31]:
df_aux = pd.DataFrame(data_aux)

In [35]:
df.head()

,dataid,localhour,gen,use,pv_system_size,gen_per_kW
0,26,2017-01-01 00:00:00,0.0,1.788817,6.0,0.0
1,26,2017-01-01 01:00:00,0.0,1.223917,6.0,0.0
2,26,2017-01-01 02:00:00,0.0,0.668367,6.0,0.0
3,26,2017-01-01 03:00:00,0.0,0.478667,6.0,0.0
4,26,2017-01-01 04:00:00,0.0,0.416283,6.0,0.0
